In [2]:
import org.apache.iceberg.catalog.Catalog;
import org.apache.hadoop.conf.Configuration;
import org.apache.iceberg.CatalogProperties;
import org.apache.iceberg.rest.RESTCatalog;
import org.apache.iceberg.aws.s3.S3FileIOProperties;


Map<String, String> properties = new HashMap<>();

properties.put(CatalogProperties.CATALOG_IMPL, "org.apache.iceberg.rest.RESTCatalog");
properties.put(CatalogProperties.URI, "http://iceberg-rest:8181");
properties.put(CatalogProperties.WAREHOUSE_LOCATION, "s3a://warehouse/wh");
properties.put(CatalogProperties.FILE_IO_IMPL, "org.apache.iceberg.aws.s3.S3FileIO");
properties.put(S3FileIOProperties.ENDPOINT, "http://minio.net:9000");

RESTCatalog catalog = new RESTCatalog();
Configuration conf = new Configuration();
catalog.setConf(conf);
catalog.initialize("demo", properties);
// catalog.name();





In [3]:
import org.apache.iceberg.Table;
import org.apache.iceberg.TableScan;
import org.apache.iceberg.catalog.Namespace;
import org.apache.iceberg.catalog.TableIdentifier;

Namespace ns = Namespace.of("t_rule_action");
TableIdentifier table_name = TableIdentifier.of(ns, "t_rule_action");
Table table = catalog.loadTable(table_name);

System.out.println(String.format("table name: %s", table.name()));


table name: demo.t_rule_action.t_rule_action


In [19]:
import org.apache.iceberg.io.CloseableIterable;
import org.apache.iceberg.data.Record;
import org.apache.iceberg.data.IcebergGenerics;

table.refresh();

CloseableIterable<Record> result = IcebergGenerics.read(table).build();

for (Record r: result) {
    System.out.println(r);
}

Record(Mr, Nah, 912381, 4321, Mouse, mouse@email.com)
Record(Mr2, Nah2, 912381999, 11849, Mouse2, mouse2@email.com)


In [18]:
import org.apache.iceberg.TableScan;
import org.apache.iceberg.DataFile;
import org.apache.iceberg.CombinedScanTask;


TableScan scan = table.newScan();
Iterable<CombinedScanTask> result = scan.select("*").planTasks();
CombinedScanTask task = result.iterator().next();
DataFile dataFile = task.files().iterator().next().file();
System.out.println(dataFile);
System.out.println(dataFile.CONTENT);


GenericDataFile{content=data, file_path=s3://warehouse/t_rule_action/t_rule_action/data/data-file.avro, file_format=AVRO, spec_id=0, partition=PartitionData{}, record_count=2, file_size_in_bytes=2388, column_sizes=null, value_counts=null, null_value_counts=null, nan_value_counts=null, lower_bounds=null, upper_bounds=null, key_metadata=null, split_offsets=null, equality_ids=null, sort_order_id=null, data_sequence_number=6, file_sequence_number=6, referenced_data_file=null, content_offset=null, content_size_in_bytes=null}
134: content: optional int (Contents of the file: 0=data, 1=position deletes, 2=equality deletes)


In [12]:
table.refresh();

System.out.println(table.snapshots());

System.out.println(table.currentSnapshot());

System.out.println(table.currentSnapshot().manifestListLocation());

table.io();

table.currentSnapshot().allManifests(table.io())

/*
import org.apache.iceberg.avro.Avro;
import org.apache.iceberg.io.InputFile;
import org.apache.iceberg.GenericManifestFile;
import org.apache.iceberg.ManifestFile;
import java.util.List;
import org.apache.iceberg.relocated.com.google.common.collect.Lists;
*/

//InputFile deleteme = table.io().newInputFile("s3://warehouse/snap-2104694741369517975-0-742aa2a7-fb0f-411b-9875-c63058bfbc4e.avro");
//InputFile deleteme = table.io().newInputFile("s3://warehouse/snap-patch.avro");
//List<ManifestFile> l = Lists.newLinkedList(Avro.read(deleteme).project(ManifestFile.schema()).build());

//System.out.println(l);

/*
InputFile manifest_list = table.io().newInputFile(table.currentSnapshot().manifestListLocation());
for (ManifestFile f : Avro.read(manifest_list).project(ManifestFile.schema()).build()) {
  System.out.println(f);
}
*/
// List<ManifestFile> l = Lists.newLinkedList(Avro.read(manifest_list).project(ManifestFile.schema()).build());



[BaseSnapshot{id=697711801, timestamp_ms=1743511509694, operation=append, summary={}, manifest-list=s3://warehouse/t_rule_action/t_rule_action/metadata/snap.avro, schema-id=0}, BaseSnapshot{id=3358094239, timestamp_ms=1743511655812, operation=append, summary={}, manifest-list=s3://warehouse/t_rule_action/t_rule_action/metadata/snap.avro, schema-id=0}]
BaseSnapshot{id=3358094239, timestamp_ms=1743511655812, operation=append, summary={}, manifest-list=s3://warehouse/t_rule_action/t_rule_action/metadata/snap.avro, schema-id=0}
s3://warehouse/t_rule_action/t_rule_action/metadata/snap.avro


[GenericManifestFile{content=DATA, path=s3://warehouse/t_rule_action/t_rule_action/metadata/manifest-entry.avro, length=4708, partition_spec_id=0, added_snapshot_id=3358094239, added_data_files_count=1, added_rows_count=2, existing_data_files_count=0, existing_rows_count=0, deleted_data_files_count=0, deleted_rows_count=0, partitions=[], key_metadata=null, sequence_number=6, min_sequence_number=6}]

In [9]:
table.schema()

table {
  1: c_salutation: optional string
  2: c_preferred_cust_flag: optional string
  3: c_first_sales_date_sk: optional int
  4: c_customer_sk: optional int
  5: c_first_name: optional string
  6: c_email_address: optional string
}